### read data and calc returns

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data_analytics_univ_rty_2014_m.csv', parse_dates = ['DATE'])

In [ ]:
df.columns.tolist()

In [ ]:
drop_cols = [ c for c in df.columns if 'idx_return' in c ]


drop_cols1 = ['open',
 'high',
 'low',
 'close',
 'volume',
 'bid',
 'ask',
 'baspd',
 'idx_close',
 'forward_return',
 'idx_forward_return',
 'relative_return',
 'stock_excess_return']

drop_cols1 = [ c for c in drop_cols1 if c in df.columns.tolist() ]

In [ ]:

df = df.drop(columns = (drop_cols + drop_cols1))

In [ ]:
# stats and returns are as of given date
# so either shift returns up, or stats down to do regression on returns vs stats as of beginning of returns period 
# lets shift stats down to align with returns end date

In [ ]:
# shift down everything except ID, DATE, stock_return 
df.columns.tolist()

In [ ]:
shift_cols = [ c for c in df.columns if c not in ['ID', 'DATE', 'stock_return'] ]

In [ ]:
df[shift_cols] = df.groupby('ID')[shift_cols].shift(1)

### functions - rolling reg / var

### calculate simple returns 

In [ ]:
df = df.sort_values(['DATE','ID'])

In [ ]:
first_dates = df.apply(lambda x: x.first_valid_index())

first_dates = first_dates.rename('first_idx').to_frame()
value_columns = df.columns.difference(['ID', 'DATE'])

# For each column
dseries = pd.DataFrame()
for column in value_columns:
    first_date = df[df[column].notna()]['DATE'].min()
    last_date = df[df[column].notna()]['DATE'].max()
    first_dates.loc[column,'first_date'] = first_date
    first_dates.loc[column,'last_date'] = last_date

In [ ]:
first_dates

In [ ]:
df.info()

In [ ]:
fdf = df.set_index(['ID','DATE'])

In [ ]:
fdf.info()

In [ ]:
idx = pd.IndexSlice
cutoff_date = '2015-12-31'
fdf = fdf[fdf.index.get_level_values('DATE') > cutoff_date]

In [ ]:
fdf[value_columns] = fdf.groupby('DATE')[value_columns].transform(lambda x: x.fillna(x.mean()))

In [ ]:
Ycols = value_columns.difference(['stock_return'])

In [ ]:
from scipy.stats.mstats import winsorize
from scipy.stats import zscore

# df[numcols] = df[numcols].groupby('ITERATION_DATE').transform(lambda x: winsorize(x, limits = (0.01,0.01)))
fdf[Ycols] = fdf[Ycols].groupby('DATE').transform(lambda x: zscore(x).clip(-3,3))

In [ ]:
def cross_sectional_regression(df, dependent_var, independent_vars, reg_type='OLS', alpha=1.0):
    """
    Performs regression in parallel for each group using joblib, with options for OLS, Ridge, and Lasso.
    Returns np.nan for coefficients when X or y contains NaN values.

    Args:
        df: Pandas DataFrame containing the data with columns ID, DATE, and other variables
        dependent_var: Name of the dependent variable column
        independent_vars: List of names of independent variable columns
        reg_type: Type of regression to perform ('OLS', 'Ridge', 'Lasso'). Default is 'OLS'
        alpha: Regularization strength for Ridge and Lasso. Default is 1.0

    Returns:
        Pandas DataFrame with the regression coefficients for each ID and DATE
    """
    from sklearn.linear_model import LinearRegression, Ridge, Lasso
    from joblib import Parallel, delayed
    import numpy as np
    import pandas as pd
    
    def _regress_group(group_data):
        X = group_data[independent_vars].values
        y = group_data[dependent_var].values.reshape(-1,1)
        
        # Return NaN coefficients if data contains NaN
        if np.any(np.isnan(X)) or np.any(np.isnan(y)):
            return pd.Series({
                'DATE': group_data['DATE'].iloc[0],
                'intercept': np.nan,
                **dict(zip(independent_vars, [np.nan] * len(independent_vars)))
            })
            
        if reg_type.upper() == 'OLS':
            model = LinearRegression()
        elif reg_type.upper() == 'RIDGE':
            model = Ridge(alpha=alpha)
        elif reg_type.upper() == 'LASSO':
            model = Lasso(alpha=alpha)
        else:
            raise ValueError("Invalid reg_type. Choose 'OLS', 'Ridge', or 'Lasso'.")
            
        model.fit(X, y)
        coefs = model.coef_.flatten()
        intercept = model.intercept_
        
        return pd.Series({
            'DATE': group_data['DATE'].iloc[0],
            'intercept': intercept[0] if isinstance(intercept, np.ndarray) else intercept,
            **dict(zip(independent_vars, coefs))
        })
    
    # Group by DATE and perform regression
    results = Parallel(n_jobs=-1)(
        delayed(_regress_group)(group) 
        for _, group in df.groupby('DATE')
    )
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.set_index('DATE')
    
    return results_df

# Let's create an example to demonstrate the usage

In [ ]:
fdf = fdf.reset_index()

In [ ]:
betas = cross_sectional_regression(fdf, dependent_var='stock_return', independent_vars=Ycols)

In [ ]:
betas = betas.rolling(12).mean()
betas = betas.reset_index().set_index('DATE')
betas.index = pd.to_datetime(betas.index)

In [ ]:
betas_shifted = pd.concat([
    betas.shift(1),
    pd.DataFrame([betas.iloc[-1]], index=[betas.index[-1] + pd.offsets.MonthEnd(1)])
])
betas_shifted.index.name = 'DATE'

In [ ]:
# for manual confirmation 
betas_shifted[list(Ycols) + ['intercept']].to_csv('betas.csv')
fdf[['ID','DATE'] + list(Ycols)].to_csv('fdf.csv')

In [ ]:
fdf = fdf.set_index(['ID','DATE'])

In [ ]:
yfit = fdf[Ycols].assign(intercept = 1).mul(betas_shifted, level = 'DATE').sum(axis = 1).rename('exp_ret').to_frame()

In [ ]:
yfit['quantile'] = yfit.groupby('DATE', group_keys = False)['exp_ret'].apply(lambda x: pd.cut(x, 10, labels = False)).rename('quantile')

In [ ]:
# using stats as of date 
# Dec 1st stats (shifted) with Dec end returns gives Dec beta. 
# avearge of Jan to Dec betas gives Dec average beta 
# Dec beta with Dec returns gives dec ranks 

In [ ]:
yfit = yfit.merge(fdf[['stock_return']], left_index = True, right_index = True)

In [ ]:
# this is ignoring a lot of the features, but need this for now to proceed with just model fitting 
# goal is daily data, 


In [ ]:
yfit_clean = yfit[yfit.index.get_level_values('DATE')> pd.Timestamp('2016-12-31')]

In [ ]:
yfit_clean.to_csv('rty_2014_smallfeatures.csv')

In [ ]:
# what output do we need?
# 1. betas 
# 2. tstats
# 3. ranks and realized returns for decile testing => goes to rank backtester 
# 4. current portfolio 
# 5. can redo ranks based on combining top betas differently 
